In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence
from torch.nn.utils.rnn import pad_packed_sequence

from sklearn.model_selection import train_test_split

from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

import pandas as pd

from collections import Counter
from konlpy.tag import Mecab

import time

import numpy as np

In [2]:
import warnings 
warnings.simplefilter('ignore')

In [3]:
df_train = pd.read_csv("./naver_train.csv", encoding="utf-8-sig")
df_test = pd.read_csv("./naver_test.csv", encoding="utf-8-sig")

In [4]:
df_train.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터 보고 초딩 영화 줄오버 연기조차 가볍지 않구나,1
2,10265843,너무 재밓었다 그래서 보는 것을 추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬 페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어 보이기만 했던 커스...,1


In [5]:
def delete_null(data):
    data = data.dropna(axis=0).reset_index(drop=True)
    space_idx = []
    for i in range(len(data)):
        if str.isspace(data.iloc[i, 1]) == True:
            space_idx.append(i)
    data = data.drop(space_idx)
    
    return data

In [6]:
df_train, df_test = map(delete_null, [df_train, df_test])

In [7]:
trainset = np.array(df_train.drop(["id"], axis = 1))
testset = np.array(df_test.drop(["id"], axis = 1))

In [8]:
trainset, valset= train_test_split(trainset, test_size=0.1)

In [9]:
X_train = trainset[:, 0]
y_train = trainset[:, 1]
X_val = valset[:, 0]
y_val = valset[:, 1]
X_test = testset[:, 0]
y_test = testset[:, 1]

In [10]:
y_train = y_train.astype(np.int64)
y_val = y_val.astype(np.int64)
y_test = y_test.astype(np.int64)

In [11]:
class Vocabulary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0
        
    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1
    
    def __call__(self, word):
        if not word in self.word2idx:
            return self.word2idx['<unk>']
        return self.word2idx[word]

    def __len__(self):
        return len(self.word2idx)

In [12]:
m = Mecab("C:\mecab\mecab-ko-dic")

def tokenizer(text):
    return m.morphs(text)

def build_vocab(data, threshold):
    counter = Counter()

    for i in range(len(data)):
        tokens = tokenizer(data[i])
        counter.update(tokens)

    words = [word for word, cnt in counter.items() if cnt >= threshold]
    vocab = Vocabulary()
    vocab.add_word('<pad>')
    vocab.add_word('<unk>')
    for w in words:
        vocab.add_word(w)
    return vocab

def tokenizing(data, max_length=256):
    data_size = len(data)
    length = []
    for i in range(data_size):
        data[i] = tokenizer(data[i])
        length.append(len(data[i]))
        if len(data[i]) > max_length:
            data[i] = data[:max_length]
        else:
            for _ in range(max_length-len(data[i])):
                data[i].append("<pad>")
    return data, length

In [13]:
vocab = build_vocab(X_train, 1)

In [14]:
print(vocab.word2idx)

{'<pad>': 0, '<unk>': 1, '후': 2, '덜덜': 3, '오프닝': 4, '씬': 5, '이': 6, '무섭': 7, '다': 8, '편': 9, '낫': 10, '고준희': 11, '도': 12, '연기자': 13, '라고': 14, '쯧': 15, '쯔': 16, '무간도': 17, '의': 18, '명성': 19, '에': 20, '흠집': 21, '을': 22, '낸': 23, '임': 24, '최고': 25, '만들': 26, '고': 27, '이런': 28, '류': 29, '영화': 30, '로': 31, '연출': 32, '할': 33, '줄': 34, '야': 35, '순수': 36, '함': 37, '그': 38, '자체': 39, '로서': 40, '만': 41, '머물': 42, '어이없': 43, '는': 44, '시나리오': 45, '조': 46, '올라': 47, '조타': 48, '송강호': 49, '서울': 50, '말씨': 51, '정말': 52, '못': 53, '듣': 54, '겠': 55, '한국': 56, '진짜': 57, '캐릭터': 58, '소화': 59, '사람': 60, '없': 61, '을까': 62, '웃긴': 63, '썩': 64, '추천': 65, '은': 66, '하': 67, '다가': 68, '어디': 69, '서': 70, '찾': 71, '나': 72, '보': 73, '구': 74, '싶': 75, '은데': 76, '개인': 77, '적': 78, '으로': 79, '잘': 80, '모르': 81, '던': 82, '이게': 83, '면': 84, '영구': 85, '와': 86, '땡': 87, '칠': 88, '아카데미': 89, '작품상': 90, '너무': 91, '뻔해서': 92, '지루': 93, '했': 94, '지만': 95, '선': 96, '균': 97, '꿀': 98, '목소리': 99, '때문': 100, '끝': 101, '까지': 102, '았': 1

In [15]:
X_train, X_val, X_test = map(tokenizing, [X_train, X_val, X_test])

In [16]:
print(X_train[0][0])

['후', '덜덜', '오프닝', '씬', '이', '무섭', '다', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>

In [17]:
print(X_train[1][0])

7


In [18]:
def token2idx(data, vocab):
    data_size = len(data[0])
    sentence_length = len(data[0][0])
    for i in range(data_size):
        data[0][i] = [vocab(x) for x in data[0][i]]
    return data

In [19]:
X_train, X_val, X_test = map(lambda data : token2idx(data, vocab), [X_train, X_val, X_test])

In [20]:
print(X_train[0][0])

[2, 3, 4, 5, 6, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [21]:
print(X_train[1][0])

7


In [22]:
class CustomDataset(Dataset):
    def __init__(self, data, y,):
        self.x = data[0]
        self.length = data[1]
        self.y = y
        
    def __getitem__(self, index):
        return (torch.tensor(self.x[index]), self.length[index], self.y[index])
    
    def __len__(self):
        return len(self.x)

In [23]:
trainset = CustomDataset(X_train, y_train)
valset = CustomDataset(X_val, y_val)
testset = CustomDataset(X_test, y_test)

In [24]:
trainset[0]

(tensor([2, 3, 4, 5, 6, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 7,
 1)

In [25]:
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
valloader = DataLoader(valset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=True)

In [26]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print("cpu 와 cuda 중 다음 기기로 학슴함: ", DEVICE)

cpu 와 cuda 중 다음 기기로 학슴함:  cuda


In [27]:
input_file = "glove.txt"
output_file = "tmp.txt"

glove2word2vec(input_file, output_file)

glove = KeyedVectors.load_word2vec_format(output_file, binary=False)

In [28]:
vocab_size = len(vocab.word2idx.keys())
embedding_size = 100
embedding_weight = np.zeros((vocab_size, embedding_size))
for i in range(2, vocab_size):
    if vocab.idx2word[i] in glove.key_to_index.keys():
        embedding_weight[i] = glove[vocab.idx2word[i]]
embedding_weight = torch.tensor(embedding_weight)

In [29]:
print(embedding_weight)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.1392, -0.5056,  1.0618,  ...,  0.8295,  0.5887, -0.6639],
        ...,
        [-0.2332, -0.1007,  0.0878,  ...,  0.4037, -0.5654, -0.4202],
        [ 0.2265, -0.0125, -0.5600,  ..., -0.1483, -0.0452,  0.6158],
        [ 0.2966, -0.0972,  0.0357,  ..., -0.4360, -0.4773, -0.6277]],
       dtype=torch.float64)


In [44]:
model.load_state_dict(torch.load("./textclassificatior.pt"))

<All keys matched successfully>

In [45]:
test_loss, test_accuracy = evaluate(model, testloader)
print(test_accuracy)

tensor(86.3185, device='cuda:0')


In [30]:
class CNN(nn.Module):
    def __init__(self, n_vocab, embed_dim, n_classes, dropout_p = 0.2):
        super(CNN, self).__init__()
        self.embedding = nn.Embedding(n_vocab, embed_dim)
        
        self.conv_0 = nn.Conv2d(in_channels = 1, 
                                out_channels = 32, 
                                kernel_size = (3, embed_dim))
        
        self.conv_1 = nn.Conv2d(in_channels = 1, 
                                out_channels = 32, 
                                kernel_size = (4, embed_dim))
        
        self.conv_2 = nn.Conv2d(in_channels = 1, 
                                out_channels = 32, 
                                kernel_size = (5, embed_dim))
        
        self.fc = nn.Linear(3 * 32, n_classes)
        
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, x):
        embedded = self.embedding(x)
        embedded = embedded.unsqueeze(1)
        
        conved_0 = F.relu(self.conv_0(embedded).squeeze(3))
        conved_1 = F.relu(self.conv_1(embedded).squeeze(3))
        conved_2 = F.relu(self.conv_2(embedded).squeeze(3))
        
        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
        
        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2), dim = 1))
        
        return self.fc(cat)

In [31]:
n_classes = 2

model = CNN(vocab_size, 100, n_classes).to(DEVICE)
lr = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

In [32]:
def train(model, optimizer, train_iter):
    model.train()
    corrects, total_loss = 0, 0
    size = 0
    for b, batch in enumerate(train_iter):
        x , l, y = batch
        x = x.to(DEVICE)
        y = y.long().to(DEVICE)
        y = y.reshape(-1)
        optimizer.zero_grad()
        logit = model(x)
        loss = F.cross_entropy(logit, y, reduction="sum")
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
        size += x.shape[0]
    avg_loss = total_loss / size
    avg_accuracy = 100.0 * corrects / size
    return avg_loss, avg_accuracy 

In [33]:
def evaluate(model, val_iter):
    model.eval()
    corrects, total_loss = 0, 0
    size = 0
    with torch.no_grad():
        for batch in val_iter:
            x , l, y = batch
            x = x.to(DEVICE)
            y = y.long().to(DEVICE)
            y = y.reshape(-1)
            logit = model(x)
            loss = F.cross_entropy(logit, y, reduction="sum")
            total_loss += loss.item()
            corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()    
            size += x.shape[0]
    avg_loss = total_loss / size
    avg_accuracy = 100.0 * corrects / size
    return avg_loss, avg_accuracy

In [34]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [35]:
model.embedding.weight.data.copy_(embedding_weight)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.1392, -0.5056,  1.0618,  ...,  0.8295,  0.5887, -0.6639],
        ...,
        [-0.2332, -0.1007,  0.0878,  ...,  0.4037, -0.5654, -0.4202],
        [ 0.2265, -0.0125, -0.5600,  ..., -0.1483, -0.0452,  0.6158],
        [ 0.2966, -0.0972,  0.0357,  ..., -0.4360, -0.4773, -0.6277]],
       device='cuda:0')

In [36]:
best_val_loss = None
n_epochs = 15
for epoch in range(n_epochs):
    
    start_time = time.time()
    
    train_loss, train_accuracy = train(model, optimizer, trainloader)
    val_loss, val_accuracy = evaluate(model, valloader)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_accuracy:.2f}%')
    print(f'\t Val. Loss: {val_loss:.3f} |  Val. Acc: {val_accuracy:.2f}%')
    
    if not best_val_loss or val_loss < best_val_loss:
        torch.save(model.state_dict(), "./textclassificatior.pt")
        best_val_loss = val_loss

Epoch: 01 | Epoch Time: 1m 36s
	Train Loss: 0.482 | Train Acc: 77.24%
	 Val. Loss: 0.406 |  Val. Acc: 81.64%
Epoch: 02 | Epoch Time: 1m 26s
	Train Loss: 0.387 | Train Acc: 82.73%
	 Val. Loss: 0.377 |  Val. Acc: 83.23%
Epoch: 03 | Epoch Time: 1m 26s
	Train Loss: 0.357 | Train Acc: 84.49%
	 Val. Loss: 0.363 |  Val. Acc: 83.83%
Epoch: 04 | Epoch Time: 1m 25s
	Train Loss: 0.335 | Train Acc: 85.65%
	 Val. Loss: 0.353 |  Val. Acc: 84.68%
Epoch: 05 | Epoch Time: 1m 25s
	Train Loss: 0.316 | Train Acc: 86.66%
	 Val. Loss: 0.347 |  Val. Acc: 84.89%
Epoch: 06 | Epoch Time: 1m 25s
	Train Loss: 0.301 | Train Acc: 87.46%
	 Val. Loss: 0.342 |  Val. Acc: 85.25%
Epoch: 07 | Epoch Time: 1m 25s
	Train Loss: 0.286 | Train Acc: 88.19%
	 Val. Loss: 0.340 |  Val. Acc: 85.47%
Epoch: 08 | Epoch Time: 1m 26s
	Train Loss: 0.273 | Train Acc: 88.84%
	 Val. Loss: 0.340 |  Val. Acc: 85.45%
Epoch: 09 | Epoch Time: 1m 26s
	Train Loss: 0.260 | Train Acc: 89.45%
	 Val. Loss: 0.341 |  Val. Acc: 85.31%
Epoch: 10 | Epoch T

In [37]:
model.load_state_dict(torch.load("./textclassificatior.pt"))

<All keys matched successfully>

In [41]:
test_loss, test_accuracy = evaluate(model, testloader)
print(test_accuracy)

tensor(85.4004, device='cuda:0')
